In [5]:
import cudf
import dask.dataframe as dd
import dask
dask.config.set({"dataframe.backend": "cudf"})


price_data = dd.read_csv('/home/edoardocame/Desktop/python_dir/eurgbp1m.csv')
price_data['sma'] = price_data['close'].rolling(60).mean()
price_data['upper_band'] = price_data['sma'] + price_data['close'].rolling(60).std() * 5
price_data['lower_band'] = price_data['sma'] - price_data['close'].rolling(60).std() * 5
price_data.tail()

,timestamp,open,high,low,close,volume,sma,upper_band,lower_band
237678,2024-08-21 23:55:00,0.85204,0.85210,0.85204,0.85209,35.610001,0.852075,0.852345,0.851805
237679,2024-08-21 23:56:00,0.85209,0.85210,0.85204,0.85205,30.100000,0.852075,0.852344,0.851807
237680,2024-08-21 23:57:00,0.85205,0.85205,0.85203,0.85203,13.250000,0.852075,0.852344,0.851807
237681,2024-08-21 23:58:00,0.85203,0.85208,0.85200,0.85207,25.340000,0.852076,0.852343,0.851809
237682,2024-08-21 23:59:00,0.85206,0.85206,0.85199,0.85200,11.280000,0.852075,0.852346,0.851805


In [ ]:
#PROTO-LOGIC BEHIND THE BOLLINGER BANDS PANDAS VERSION.



# Initialize positions
price_data['positions'] = 0

# Generate signals
price_data['long_entry'] = (price_data['price'] < price_data['lower_band'])
price_data['long_exit'] = (price_data['price'] > price_data['middle_band'])
price_data['short_entry'] = (price_data['price'] > price_data['upper_band'])
price_data['short_exit'] = (price_data['price'] < price_data['middle_band'])

# Create position column
price_data.loc[price_data['long_entry'], 'positions'] = 1
price_data.loc[price_data['long_exit'], 'positions'] = 0
price_data.loc[price_data['short_entry'], 'positions'] = -1
price_data.loc[price_data['short_exit'], 'positions'] = 0

# Forward fill positions
price_data['positions'] = price_data['positions'].fillna(method='ffill').fillna(0)

# Record entry price
price_data['entry_price'] = price_data['price'].where(price_data['positions'].diff() != 0).fillna(method='ffill')

# Set stop loss percentage
stop_loss_percentage = 0.05
# Apply stop loss using cuprice_data
price_data.loc[(price_data['positions'] == 1) & (price_data['price'] < price_data['entry_price'] * (1 - stop_loss_percentage)), 'positions'] = 0
price_data.loc[(price_data['positions'] == -1) & (price_data['price'] > price_data['entry_price'] * (1 + stop_loss_percentage)), 'positions'] = 0

# Recalculate positions after stop loss
price_data['positions'] = price_data['positions'].fillna(method='ffill').fillna(0)

# Calculate returns
price_data['market_returns'] = price_data['price'].pct_change()
price_data['strategy_returns'] = price_data['market_returns'] * price_data['positions'].shift(1)